# Cafe Dataset
## Data Cleaning
by: Mark June Almojuela

Last Modified: 11-10-2025

This activity uses python's Pandas for data cleaning.

Data Source: https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training
(Kaggle)

In [1]:
# venv activation
!python --version
!pip install pandas

Python 3.13.2



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Package imports
import pandas as pd
from pprint import pprint

#### Initial Quality Check

In [3]:
df = pd.read_csv('../data/raw/dirty_cafe_sales.csv')

pprint(df.info())
pprint(df.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
None
Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object


### A. Transaction ID Check

In [4]:
print('Unique Transaction IDs:', df['Transaction ID'].nunique())
print(df['Transaction ID'].head())

# Notably, there 1000 entries with unique IDS. These IDS start with the prefix 'TXN' followed by a sequence of numbers.

invalid_ids = df[~df['Transaction ID'].str.startswith('TXN_')]
print('Invalid ID Count:', invalid_ids['Transaction ID'].count())

df['Transaction ID'] = df['Transaction ID'].astype(str)

# There are no invalid Transaction IDs in the dataset. All Transaction IDs conform to the expected format.

Unique Transaction IDs: 10000
0    TXN_1961373
1    TXN_4977031
2    TXN_4271903
3    TXN_7034554
4    TXN_3160411
Name: Transaction ID, dtype: object
Invalid ID Count: 0


### B. Item, Quantity, Price Check

#### B.0 Initial Checks

In [5]:
print('Unique Items:', df['Item'].value_counts())

Unique Items: Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64


In [6]:
error_items = df[df['Item'] == 'ERROR']
print('Error Transaction Count:', error_items.shape[0])
print('Error Transactions:', error_items.head())

Error Transaction Count: 292
Error Transactions:     Transaction ID   Item Quantity Price Per Unit Total Spent  Payment Method  \
14     TXN_8915701  ERROR        2            1.5         3.0             NaN   
69     TXN_8471743  ERROR        5            3.0        15.0  Digital Wallet   
88     TXN_9487821  ERROR        1            5.0         5.0  Digital Wallet   
118    TXN_4633784  ERROR        5            NaN        15.0             NaN   
211    TXN_6717827  ERROR        3            5.0        15.0  Digital Wallet   

     Location Transaction Date  
14   In-store       2023-03-21  
69   In-store       2023-04-06  
88   Takeaway       2023-05-24  
118  In-store       2023-02-06  
211       NaN       2023-11-15  


In [7]:
unknown_items = df[df['Item'] == 'UNKNOWN']
print('Unknown Transaction Count:', unknown_items.shape[0])
print('Unknown Transactions:\n', unknown_items.head())

Unknown Transaction Count: 344
Unknown Transactions:
    Transaction ID     Item Quantity Price Per Unit Total Spent  \
6     TXN_4433211  UNKNOWN        3            3.0         9.0   
31    TXN_8927252  UNKNOWN        2            1.0       ERROR   
33    TXN_7710508  UNKNOWN        5            1.0         5.0   
36    TXN_6855453  UNKNOWN        4            3.0        12.0   
52    TXN_8914892  UNKNOWN        5            5.0        25.0   

    Payment Method  Location Transaction Date  
6            ERROR  Takeaway       2023-10-06  
31     Credit Card     ERROR       2023-11-06  
33            Cash       NaN            ERROR  
36             NaN  In-store       2023-07-17  
52  Digital Wallet       NaN       2023-03-15  


In [8]:
item_price = df[['Item', 'Price Per Unit']]
print(item_price.value_counts().sort_index())

Item      Price Per Unit
Cake      3.0               1085
          ERROR               19
          UNKNOWN             14
Coffee    2.0               1108
          ERROR               18
          UNKNOWN             20
Cookie    1.0               1026
          ERROR               21
          UNKNOWN             21
ERROR     1.0                 34
          1.5                 37
          2.0                 31
          3.0                 77
          4.0                 61
          5.0                 39
          ERROR                3
          UNKNOWN              3
Juice     3.0               1110
          ERROR               26
          UNKNOWN             18
Salad     5.0               1082
          ERROR               34
          UNKNOWN             16
Sandwich  4.0               1082
          ERROR               13
          UNKNOWN             19
Smoothie  4.0               1036
          ERROR               19
          UNKNOWN             17
Tea       1.5     

In [9]:
# Check for invalid quantity values
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce') 
invalid_quantities = df[df['Quantity'].isna()]
print('Invalid Quantity Count:', invalid_quantities.shape[0])

Invalid Quantity Count: 479


In [10]:
# Check for invalid Price Per Unit values
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
invalid_price_per_unit = df[df['Price Per Unit'].isna()]
print('Invalid Price Per Unit Count:', invalid_price_per_unit.shape[0])

Invalid Price Per Unit Count: 533


In [11]:
# Check for invalid Total Spent values
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
invalid_totals = df[df['Total Spent'].isna()]
print('Invalid Total Spent Count:', invalid_totals.shape[0])

Invalid Total Spent Count: 502


#### B.1 Perform price checks

In [12]:
# Corrected 'Price Per Unit' entries based on the observed 'Item' values.
prices = {
    'Cake': 3.0,
    'Coffee': 2.0,
    'Cookie': 1.0,
    'Juice': 3.0,
    'Salad': 5.0,
    'Sandwich': 4.0,
    'Smoothie': 4.0,
    'Tea': 1.5,
}

for item, price in prices.items():
    df.loc[(df['Item'] == item) & (df['Price Per Unit'] != price), 'Price Per Unit'] = price
item_price = df[['Item', 'Price Per Unit']]
print(item_price.value_counts().sort_index())

Item      Price Per Unit
Cake      3.0               1139
Coffee    2.0               1165
Cookie    1.0               1092
ERROR     1.0                 34
          1.5                 37
          2.0                 31
          3.0                 77
          4.0                 61
          5.0                 39
Juice     3.0               1171
Salad     5.0               1148
Sandwich  4.0               1131
Smoothie  4.0               1096
Tea       1.5               1089
UNKNOWN   1.0                 45
          1.5                 40
          2.0                 49
          3.0                 77
          4.0                 70
          5.0                 45
Name: count, dtype: int64


In [13]:
# Verify the corrections on prices 
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
invalid_price_per_unit = df[df['Price Per Unit'].isna()]
print('Invalid Price Per Unit Count:', invalid_price_per_unit.shape[0])

# Result: There are 54 invalid 'Price Per Unit' entries after correction.

Invalid Price Per Unit Count: 54


In [14]:
# Display some of the invalid Price Per Unit entries
invalid_price_per_unit.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
118,TXN_4633784,ERROR,5.0,NaN,15.0,NaN,In-store,2023-02-06
151,TXN_4031509,NaN,4.0,NaN,16.0,Credit Card,Takeaway,2023-01-04
289,TXN_3495950,NaN,4.0,NaN,6.0,Credit Card,In-store,2023-02-19
334,TXN_2523298,NaN,4.0,NaN,6.0,ERROR,In-store,2023-03-25
550,TXN_4186681,ERROR,4.0,NaN,6.0,Digital Wallet,NaN,2023-05-24


In [15]:
# Fill in the missing prices based on the total spent and quantity
for index, row in invalid_price_per_unit.iterrows():
    if pd.isna(row['Price Per Unit']) and not pd.isna(row['Total Spent']) and not pd.isna(row['Quantity']) and row['Quantity'] != 0:
        df.at[index, 'Price Per Unit'] = row['Total Spent'] / row['Quantity']

invalid_price_per_unit = df[df['Price Per Unit'].isna()]
print('Invalid Price Per Unit Count after filling:', invalid_price_per_unit.shape[0])

Invalid Price Per Unit Count after filling: 6


In [16]:
invalid_price_per_unit.head(10)

# The correction has reduced the invalid 'Price Per Unit' entries to 6. Further investigation may be needed for these remaining entries.

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
3779,TXN_7376255,UNKNOWN,NaN,NaN,25.0,NaN,In-store,2023-05-27
4152,TXN_9646000,ERROR,2.0,NaN,NaN,NaN,In-store,2023-12-14
7597,TXN_1082717,ERROR,NaN,NaN,9.0,Digital Wallet,In-store,2023-12-13
9819,TXN_1208561,NaN,NaN,NaN,20.0,Credit Card,NaN,2023-08-19


#### B.2 Perform Quantity Checks

In [17]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df = df[~df['Quantity'].isna()].copy()
print('Invalid Quantity Count after removal:', df['Quantity'].isna().sum())

Invalid Quantity Count after removal: 0


In [18]:
invalid_quantities = df[df['Quantity'].isna() | (df['Quantity'] <= 0)]
print('Invalid Quantity Count after removal:', invalid_quantities.shape[0])

Invalid Quantity Count after removal: 0


#### B.3 Item Check

In [19]:
# Fill in the missing Item names based on the Price Per Unit
for item, price in prices.items():
    df.loc[(df['Price Per Unit'] == price) & (df['Item'].isna() | (df['Item'].isin(['ERROR', 'UNKNOWN']))), 'Item'] = item
df['Item'] = df['Item'].astype(str)
df = df[df['Item'].notna()].copy()

invalid_items = df[df['Item'].isna() | (df['Item'].isin(['ERROR', 'UNKNOWN']))]
print('Invalid Item Count after filling:', invalid_items.shape[0])

Invalid Item Count after filling: 2


In [20]:
invalid_items.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
4152,TXN_9646000,ERROR,2.0,NaN,NaN,NaN,In-store,2023-12-14


#### B.4 Total Spent Check

In [21]:
# Check the Total Spent values based on the correct Price Per Unit and Quantity
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
invalid_totals = df[df['Total Spent'].isna()]
print('Invalid Total Spent Count before recalculation:', invalid_totals.shape[0])

incorrect_totals = df[df['Total Spent'] != (df['Price Per Unit'] * df['Quantity'])]
print('Incorrect Total Spent Count before recalculation:', incorrect_totals.shape[0])   

Invalid Total Spent Count before recalculation: 482
Incorrect Total Spent Count before recalculation: 482


In [22]:
# Correct the Total Spent values
df['Total Spent'] = df['Price Per Unit'] * df['Quantity']

invalid_totals = df[df['Total Spent'].isna()]
print('Invalid Total Spent Count after recalculation:', invalid_totals.shape[0])

incorrect_totals = df[df['Total Spent'] != (df['Price Per Unit'] * df['Quantity'])]
print('Incorrect Total Spent Count after recalculation:', incorrect_totals.shape[0])

# There are 3 invalid and incorrect Total Spent entries after recalculation.

Invalid Total Spent Count after recalculation: 3
Incorrect Total Spent Count after recalculation: 3


In [23]:
invalid_totals.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1761,TXN_3611851,nan,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
4152,TXN_9646000,ERROR,2.0,NaN,NaN,NaN,In-store,2023-12-14


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9521 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9521 non-null   object 
 1   Item              9521 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9518 non-null   float64
 4   Total Spent       9518 non-null   float64
 5   Payment Method    7074 non-null   object 
 6   Location          6412 non-null   object 
 7   Transaction Date  9371 non-null   object 
dtypes: float64(3), object(5)
memory usage: 669.4+ KB


In [25]:
df = df[df['Item'].notna() & ~df['Item'].isin(['ERROR', 'UNKNOWN', 'nan'])].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9518 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9518 non-null   object 
 1   Item              9518 non-null   object 
 2   Quantity          9518 non-null   float64
 3   Price Per Unit    9518 non-null   float64
 4   Total Spent       9518 non-null   float64
 5   Payment Method    7072 non-null   object 
 6   Location          6411 non-null   object 
 7   Transaction Date  9368 non-null   object 
dtypes: float64(3), object(5)
memory usage: 669.2+ KB


#### Section B Validations

In [26]:
df['Item'].value_counts()

Item
Cake        1333
Sandwich    1280
Coffee      1218
Salad       1214
Cookie      1156
Tea         1141
Juice       1124
Smoothie    1052
Name: count, dtype: int64

In [27]:
invalid_items = df[df['Item'].isna() | (df['Item'].isin(['ERROR', 'UNKNOWN', 'nan']))]
print('Invalid Item Count after filling:', invalid_items.shape[0])

invalid_quantities = df[df['Quantity'].isna() | (df['Quantity'] <= 0)]
print('Invalid Quantity Count after removal:', invalid_quantities.shape[0])

invalid_price_per_unit = df[df['Price Per Unit'].isna() | (df['Price Per Unit'] <= 0)]
print('Invalid Price Per Unit Count after filling:', invalid_price_per_unit.shape[0])

invalid_totals = df[df['Total Spent'].isna() | (df['Total Spent'] != (df['Price Per Unit'] * df['Quantity']))]
print('Invalid Total Spent Count after recalculation:', invalid_totals.shape[0])

Invalid Item Count after filling: 0
Invalid Quantity Count after removal: 0
Invalid Price Per Unit Count after filling: 0
Invalid Total Spent Count after recalculation: 0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9518 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9518 non-null   object 
 1   Item              9518 non-null   object 
 2   Quantity          9518 non-null   float64
 3   Price Per Unit    9518 non-null   float64
 4   Total Spent       9518 non-null   float64
 5   Payment Method    7072 non-null   object 
 6   Location          6411 non-null   object 
 7   Transaction Date  9368 non-null   object 
dtypes: float64(3), object(5)
memory usage: 669.2+ KB


### C. Payment, Location, Date

#### C.0 Initial Check

In [29]:
pprint('Location Value Counts:')
df['Location'].value_counts()

'Location Value Counts:'


Location
In-store    2877
Takeaway    2870
ERROR        341
UNKNOWN      323
Name: count, dtype: int64

In [30]:
pprint('Payment Method Value Counts:')
df['Payment Method'].value_counts()

'Payment Method Value Counts:'


Payment Method
Digital Wallet    2182
Credit Card       2168
Cash              2155
ERROR              288
UNKNOWN            279
Name: count, dtype: int64

In [31]:
df['Transaction Date'].head()
# Transaction Date column follows the format 'YYYY-MM-DD'

0    2023-09-08
1    2023-05-16
2    2023-07-19
3    2023-04-27
4    2023-06-11
Name: Transaction Date, dtype: object

In [32]:
invalid_dates = df[pd.to_datetime(df['Transaction Date'], errors='coerce').isna()]

print('Invalid Transaction Date Count:', invalid_dates.shape[0])
# There are 435 invalid dates

Invalid Transaction Date Count: 435


#### C.1 Transaction Date Cleanup

In [33]:
invalid_dates.sample(10)
# The invalid dates appear to be caused by errors, missing, and unknown entries

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
749,TXN_6916047,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,ERROR
7100,TXN_5468350,Salad,4.0,5.0,20.0,Credit Card,In-store,NaN
6298,TXN_9868954,Smoothie,4.0,4.0,16.0,Credit Card,NaN,NaN
7009,TXN_4558455,Salad,3.0,5.0,15.0,Credit Card,In-store,NaN
259,TXN_5003018,Cookie,4.0,1.0,4.0,Digital Wallet,NaN,UNKNOWN
5499,TXN_2268203,Salad,3.0,5.0,15.0,NaN,UNKNOWN,UNKNOWN
3327,TXN_7990902,Coffee,5.0,2.0,10.0,ERROR,In-store,ERROR
2804,TXN_6228918,Cake,1.0,3.0,3.0,Digital Wallet,In-store,UNKNOWN
1033,TXN_9218683,Cake,2.0,3.0,6.0,Digital Wallet,In-store,ERROR
9676,TXN_2827259,Tea,3.0,1.5,4.5,Credit Card,Takeaway,NaN


In [34]:
# Cleanup the transaction date
df['Transaction Date'] = df['Transaction Date'].fillna('UNKNOWN')
df['Transaction Date'] = df['Transaction Date'].replace('', 'UNKNOWN')

I kept the ERROR and UKNOWN values in the location and payment method columns as they might represent valid data points indicating missing or unspecified information.

Meanwhile, the unknown Item values poised a very unusable state due to lost price and quantity values which are most crucial for analytics. 

### Export

In [36]:
df.to_csv('../data/cleaned/clean_cafe_sales.csv', index=False)